# Prelim Feature Engineering: User
Feature engineering done *before* splitting train/test. This is only for our viewing and analysis, should not be used in the formal modelling.

In [1]:
import pandas as pd

df = pd.read_csv('../00_dataset/YelpZip/metadata', 
                 sep='\t',
                 header=None,
                 names=['user_id', 'restaurant_id', 'user_rating', 'label', 'date'])

# reviews_df = pd.read_csv('../00_dataset/reviewContent',
#                 sep='\t',
#                 header=None,
#                 names=['user_id', 'restaurant_id', 'date', 'review'])

In [ ]:
# df = df.merge(reviews_df,
#               left_on=['user_id', 'restaurant_id', 'date'],
#               right_on=['user_id', 'restaurant_id', 'date'],
#               how='left')

In [2]:
df

,user_id,restaurant_id,user_rating,label,date
0,5044,0,1.0,-1,2014-11-16
1,5045,0,1.0,-1,2014-09-08
2,5046,0,3.0,-1,2013-10-06
3,5047,0,5.0,-1,2014-11-30
4,5048,0,5.0,-1,2014-08-28
...,...,...,...,...,...
608593,119664,5039,4.0,1,2013-01-20
608594,56277,5039,2.0,1,2012-11-12
608595,265320,5039,1.0,1,2012-08-22
608596,161722,5039,4.0,1,2011-05-11


In [3]:
print(f"""
Unique users: {df['user_id'].nunique()},
Unique restaraunts: {df['restaurant_id'].nunique()}
""")

# Get count of restaurants per user
user_restaurant_counts = df.groupby('user_id')['restaurant_id'].nunique()

# Percentage of users who reviewed more than X number of restaurants
limiters = [10, 25, 50, 100, 200]
for limiter in limiters:
       multiple_reviews = (user_restaurant_counts > limiter).mean() * 100
       print(f"\n{multiple_reviews:.1f}% of users ({(user_restaurant_counts > limiter).sum()}) reviewed more than {limiter} restaurants")

# Specific users with most reviews:
# print("\nTop users by number of restaurants reviewed:")
# top_users = user_restaurant_counts.sort_values(ascending=False).head(500)
# print(top_users)

# Group by user_id and check if all labels are -1
fake_users = (df.groupby('user_id')['label']
              .agg(lambda x: (x == -1).all() and len(x) > 0)
              .sum())
real_users = (df.groupby('user_id')['label']
              .agg(lambda x: (x == 1).all() and len(x) > 0)
              .sum())

# number of fake users
print(f"\nNumber of fake users: {fake_users}, Number of real users: {real_users}, Mixed: {df['user_id'].nunique() - fake_users - real_users}")


Unique users: 260277,
Unique restaraunts: 5044


2.8% of users (7336) reviewed more than 10 restaurants

0.7% of users (1805) reviewed more than 25 restaurants

0.1% of users (375) reviewed more than 50 restaurants

0.0% of users (38) reviewed more than 100 restaurants

0.0% of users (0) reviewed more than 200 restaurants

Number of fake users: 60107, Number of real users: 198049, Mixed: 2121


## Feature Engineering

In [4]:
# Ensure 'date' column is in datetime format
df['date'] = pd.to_datetime(df['date'])

# Aggregate user features
user_features = df.groupby('user_id').agg({
    'restaurant_id': 'count',  # no. of restaurant reviews per user
    'user_rating': ['mean', 'min', 'max', 'std'],  # rating statistics
    'date': ['min', 'max']  # First and last review dates
})

# unique days active (no. of days the user has made a rating/review)
user_activity = df.groupby('user_id')['date'].nunique()

user_features[('unique_days_active', '')] = user_activity

# Calculate review timespan
user_features['review_timespan'] = (user_features[('date', 'max')] - user_features[('date', 'min')]).dt.days

# Avoid division by zero for users with only one review
user_features['review_timespan'] = user_features['review_timespan'].replace(0, 1)

# Compute average reviews per day
user_features['avg_reviews_per_day'] = user_features[('restaurant_id', 'count')] / user_features['review_timespan']

# Compute percentage of active days against user existence date
user_features['user_active_percentage'] = user_features['unique_days_active'] / user_features['review_timespan']

# round floats to 3dp
user_features = user_features.round(3)

# flatten df
user_features.columns = ['_'.join(col).strip('_') if isinstance(col, tuple) else col for col in user_features.columns]
user_features.fillna(0)

# Rename columns
user_features.rename(columns={
    'restaurant_id_count': 'user_restaurants_reviewed', 
    'date_min': 'user_earliest', 
    'date_max': 'user_latest',
    'review_timespan': 'user_review_timespan',
    'unique_days_active': 'user_days_active',
    'avg_reviews_per_day': 'users_avg_per_day'
}, inplace=True)

# keep `user_id` as a column
user_features = user_features.reset_index()

user_features

,user_id,user_restaurants_reviewed,user_rating_mean,user_rating_min,user_rating_max,user_rating_std,user_earliest,user_latest,user_days_active,user_review_timespan,users_avg_per_day,user_active_percentage
0,5044,1,1.00,1.0,1.0,NaN,2014-11-16,2014-11-16,1,1,1.000,1.000
1,5045,1,1.00,1.0,1.0,NaN,2014-09-08,2014-09-08,1,1,1.000,1.000
2,5046,4,3.25,3.0,4.0,0.5,2013-10-01,2013-11-02,4,32,0.125,0.125
3,5047,1,5.00,5.0,5.0,NaN,2014-11-30,2014-11-30,1,1,1.000,1.000
4,5048,1,5.00,5.0,5.0,NaN,2014-08-28,2014-08-28,1,1,1.000,1.000
...,...,...,...,...,...,...,...,...,...,...,...,...
260272,265316,1,5.00,5.0,5.0,NaN,2014-12-24,2014-12-24,1,1,1.000,1.000
260273,265317,1,5.00,5.0,5.0,NaN,2014-12-31,2014-12-31,1,1,1.000,1.000
260274,265318,1,4.00,4.0,4.0,NaN,2015-01-02,2015-01-02,1,1,1.000,1.000
260275,265319,1,5.00,5.0,5.0,NaN,2014-11-07,2014-11-07,1,1,1.000,1.000


## This part is just for manual analysis, do not use real/fake label count as part of features

In [ ]:
user_features[['fake_review_count', 'real_review_count']] = df.groupby('user_id')['label'] \
    .agg(fake_review_count=lambda x: (x == -1).sum(), real_review_count=lambda x: (x == 1).sum()) \
    .reset_index(drop=True)

# % of real reviews
user_features['real_review_pct'] = user_features['real_review_count'] / \
                                        (user_features['fake_review_count'] + user_features['real_review_count'])

# round to 3dp; fill with 0 if users has no real review
user_features['real_review_pct'] = user_features['real_review_pct'].fillna(0)

# sort by descending to see more complex values, but mostly real users 
user_features = user_features.sort_values(by=('real_review_count'), ascending=False)
user_features

,user_id,user_restaurants_reviewed,user_rating_mean,user_rating_min,user_rating_max,user_rating_std,user_earliest,user_latest,user_days_active,user_review_timespan,users_avg_per_day,user_active_percentage,fake_review_count,real_review_count,real_review_pct
3323,8367,197,3.980,2.0,5.0,0.851,2008-04-08,2014-12-01,196,2428,0.081,0.081,0,197,1.0
4457,9501,185,4.368,2.0,5.0,0.664,2008-05-05,2015-01-06,185,2437,0.076,0.076,0,185,1.0
2827,7871,178,3.854,1.0,5.0,0.760,2010-04-10,2014-12-28,160,1723,0.103,0.093,0,178,1.0
3181,8225,159,4.208,1.0,5.0,0.738,2007-07-14,2015-01-01,154,2728,0.058,0.056,0,159,1.0
5890,10934,155,4.161,2.0,5.0,0.650,2008-10-10,2014-12-03,135,2245,0.069,0.060,0,155,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75308,80352,1,2.000,2.0,2.0,NaN,2012-04-17,2012-04-17,1,1,1.000,1.000,1,0,0.0
125415,130459,1,5.000,5.0,5.0,NaN,2009-12-29,2009-12-29,1,1,1.000,1.000,1,0,0.0
75306,80350,4,3.750,2.0,5.0,1.258,2013-01-05,2013-01-05,1,1,4.000,1.000,4,0,0.0
75305,80349,1,2.000,2.0,2.0,NaN,2013-08-12,2013-08-12,1,1,1.000,1.000,1,0,0.0


In [6]:
view = user_features.sort_values(by='fake_review_count', ascending=False).head(20)
view

,user_id,user_restaurants_reviewed,user_rating_mean,user_rating_min,user_rating_max,user_rating_std,user_earliest,user_latest,user_days_active,user_review_timespan,users_avg_per_day,user_active_percentage,fake_review_count,real_review_count,real_review_pct
9837,14881,47,4.596,1.0,5.0,0.825,2013-11-11,2014-12-08,28,392,0.120,0.071,47,0,0.000000
7214,12258,43,4.419,1.0,5.0,1.096,2013-11-04,2014-12-08,27,399,0.108,0.068,43,0,0.000000
5434,10478,42,3.381,2.0,4.0,0.623,2013-04-29,2014-07-06,6,433,0.097,0.014,42,0,0.000000
7220,12264,40,4.425,1.0,5.0,0.958,2013-11-04,2014-12-08,23,399,0.100,0.058,40,0,0.000000
13583,18627,39,4.385,3.0,5.0,0.590,2013-05-08,2014-10-06,35,516,0.076,0.068,39,0,0.000000
4131,9175,38,4.421,1.0,5.0,0.858,2013-11-04,2014-12-08,24,399,0.095,0.060,38,0,0.000000
8167,13211,36,4.056,1.0,5.0,0.924,2012-12-28,2014-12-23,31,725,0.050,0.043,36,0,0.000000
7231,12275,35,4.229,2.0,5.0,0.770,2013-06-13,2014-12-06,16,541,0.065,0.030,35,0,0.000000
8352,13396,34,4.706,3.0,5.0,0.524,2013-11-04,2014-11-13,23,374,0.091,0.061,34,0,0.000000
29642,34686,34,4.147,1.0,5.0,1.132,2014-02-02,2014-12-08,19,309,0.110,0.061,34,0,0.000000
